In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Meshing RVE function

In [ ]:
def mesh(x,y):

    # returns [XYZ, CON, DOF] 
    # XYZ - array of nodal coordinates within RVE [nel x 2]
    # CON - array of node numbers for elements (linear QUADS [nel x 4])
    # DOF - array of element DOFs (4-node (linear) quadrilateral element => [nel x 8]); 
    
    # number of nodes and element in the RVE
    nnodesx = len(x)                     # number of horizontal nodes
    nnodesy = len(y)                    # number of vertical nodes 
    nelx = nnodesx-1                       # number of horizontal elements
    nely = nnodesy-1                       # number of vertical elements
    nnodes = nnodesx*nnodesy               # total number of nodes    

    # length of RVE
    lx = x[nnodesx-1] - x[0]           # length of RVE in x-direction (horizontal)
    ly = y[nnodesy-1] - y[0]         # length of RVE in y-direction (vertical)

    # GENERATE COORDINATES OF NODES 'XYZ'
    XYZ = np.zeros((nnodes,2))
    for i in range(nnodesy):
        for j in range(nnodesx):
            XYZ[j+i*nnodesx, :] = [x[j],y[i]]

    # NODE NUMBERS FOR ELEMENTS 
    nel = nelx*nely                              # total number of elements in RVE
    CON = np.zeros((nel,4), dtype=int)           # [nel*4] array of node number for each element
    for i in range(nely):                        # loop over elements in the vertical direction 
        for j in range(nelx):                    # loop over elements in the horizontal direction 
            # element 'el' and corresponding node numbers
            CON[j+i*nelx, :] = [j+i*nnodesx, j+i*nnodesx+1,j+(i+1)*nnodesx+1, j+(i+1)*nnodesx] 

    # Global DOF for each element (4-node (linear) quadrilateral element)
    DOF = np.zeros((nel,2*4), dtype=int)
    for i in range(nel):
        # defines single row of DOF for each element 'i'
        DOF[i,:] = [CON[i,0]*2, CON[i,1]*2-1, CON[i,1]*2, CON[i,1]*2+1,CON[i,2]*2, CON[i,2]*2+1, CON[i,3]*2, CON[i,3]*2+1]
        
    return XYZ, CON, DOF

# Plane Strain

In [ ]:
def plane_strain(E,nu):
    
    D=np.zeros((3*len(E),3))                                           #elasticity matrix - DIM: [3*element number X 3]
    
    # Your code here
    raise NotImplementedError
    
    return D

# Apply Boundary Constraints (Fixed corner nodes and force on left boundary)

In [ ]:
# Function which returns global DOFs given fixed corner nodes, and a list of internal left boundary nodes
def BC_fun(nxnodes,nynodes,ndofs):
    
    # Identify corner nodes
    # YOUR CODE HERE
    corner0 = ...
    corner1 = ...
    corner2 = ...
    corner3 = ...
    corners = np.array([corner0,corner1,corner2,corner3])

    # Corner DOFs
    cornerDOFs = np.r_[corners*2,corners*2+1]

    # Get array of DOFs after removing corners
    BCcorner = np.arange(ndofs)
    BCcorner = np.setdiff1d(BCcorner,cornerDOFs)

    # Get left internal boundary nodes
    # YOUR CODE HERE
    leftnodes = ...
    
    # Force vector
    fl = 50
    f = np.zeros((ndofs,1))
    f[2*leftnodes] = fl
    f = f[BCcorner] # Remove corners
    
    return BCcorner, f

# Strain-displacement matrix 

In [ ]:
def dispstrain_B(xyze,xi,eta):
    
    natcoord = np.array([[-1, 1, 1, -1],[-1, -1, 1, 1]])    #natural nodal coordinates of a quad element

    # derivatives of shape functions w.r.t. natural coordinates 
    dNdnat = np.zeros((2,4))
    # YOUR CODE HERE - Implement shape function derivatives

    # element Jacobian matrix
    Jmat = np.dot(dNdnat,xyze)
    J = np.linalg.det(Jmat)                          #determinant of the Jacobian
                
    JmatInv = np.linalg.inv(Jmat)                   #inverse of the Jacobian matrix
    dNdx = np.dot(JmatInv,dNdnat)                   #effectively: Jmat^-1 * dNdna
                
    #displacement-strain matrix 
    #linear QUAD element
    dsB=np.zeros((3,8))                  #[3 strain components X 8 DOFs]
    # YOUR CODE HERE - Implement displacement-strain matrix

    return dsB, J

# Element stiffness matrix

In [ ]:
def keval(xyze,De,th):
    
    ke = np.zeros((8,8))                                # create element stiffness matrix (array)                    

    a = 1/(np.sqrt(3))                                  # location of Gauss points (in natural coordinates)
    w = 1                                               # weights
    Gauss = np.array([[-a, a, a, -a],[-a, -a, a, a]])   # Gauss points matrix
        
    for i in range(4):                                # introduce natural coordinates 
        xi = Gauss[0,i]                                 # natural coordinate - horizontal  
        eta = Gauss[1,i]                                # natural coordinate - vertical 
        
        dsB, J = dispstrain_B(xyze,xi,eta);           # evaluate dsB matrix and Jacobian

        # YOUR CODE HERE - Update element stiffness matrix
        ke += ...
        
    return ke 

# Global stiffness matrix K

In [ ]:
def K_matrix(XYZ,CON,DOF,D,th,BCcorner):
    
    nel = len(CON)                        #number of elements
    ndof = 2*len(XYZ)                     #number of DOFs

    K = np.zeros((ndof,ndof))

    for i in range(nel):
        id = DOF[i,:]                      # IDs of DOFs
        xyze = XYZ[CON[i,:], :]             # element coordinates
        De = D[3*i:3*i+3, 0:3]              # elasticity matrix

        ke = keval(xyze,De,th)             # call function evaluating element stiffness matrix

        # YOUR CODE HERE - Update global stiffness matrix
        raise NotImplementedError

    K = K[np.ix_(BCcorner,BCcorner)] # Pin corners
    
    return K

# Deformation Plot

In [ ]:
def deformation_plot(XYZ,CON,d):
    ccc1=np.array(XYZ[:,0])
    ccc2=np.array(d[0:len(d):2]).reshape(-1)
    ccc= np.array(ccc1+ccc2) 
    
    ddd1=np.array(XYZ[:,1])
    ddd2=np.array(d[1:len(d):2]).reshape(-1)
    ddd= np.array(ddd1+ddd2)
    
    figure = plt.figure()
    plt.plot(XYZ[:,0], XYZ[:, 1],'sk', markersize='10')
    plt.plot(XYZ[:,0] + d[0:len(d):2].reshape(-1), XYZ[:,1] + d[1:len(d):2].reshape(-1), 'or',markersize='10')
    plt.title("Deformed RVE")
    
    for i in range(len(CON)):
        plt.fill(XYZ[CON[i, :], 0], XYZ[CON[i, :], 1], edgecolor='k', fill=False)
        plt.fill(XYZ[CON[i, :], 0] + ccc2[(CON[i, :])], XYZ[CON[i, :], 1] + ddd2[(CON[i, :])], edgecolor='r', fill=False)

# Create Mesh

In [ ]:
# Composite material with 2 phases

# Mesh coordinates in x and y directions
x = [0, 1, 2, 3]
y = [0, 1, 2, 3]
nxnodes = len(x)
nynodes = len(y)

#generate mesh
XYZ, CON, DOF = mesh(x,y)
ndofs=2*len(XYZ)

# Material properties
E = np.array([100, 300, 300, 300, 100, 300, 100, 100, 300])
nu = np.array([0.2, 0.1, 0.1, 0.1, 0.2, 0.1, 0.2, 0.2, 0.1])
th=1

In [ ]:
# Plot mesh
plt.plot(XYZ[:, 0], XYZ[:, 1], 'sk')
for i in range(len(CON)):
    plt.fill(XYZ[CON[i, :], 0], XYZ[CON[i, :], 1], edgecolor='k', fill=False)

# Compute Solution without PBCs

In [ ]:
# Elasticity matrix
D = plane_strain(E,nu)

# Get boundary conditions (fixed corners and force vector)
BCcorner, f = BC_fun(nxnodes,nynodes,ndofs)

# Stiffness matrix
K = K_matrix(XYZ,CON,DOF,D,th,BCcorner)

# Solve reduced problem
# YOUR CODE HERE - Get displacement matrix for unconstrained nodes
dm = ...

# Global displacement vector
d = np.zeros((ndofs,1))
d[BCcorner] = dm

# Plotting
deformation_plot(XYZ,CON,d)

# PBC Transformation Matrix

In [ ]:
def T_matrix(XYZ,BCcorner):

    # Number of nodes
    nnodes=len(XYZ) 

    # Boundary nodes
    small = 1e-10
    tid1=np.where(abs(XYZ[:,0]-np.amax(XYZ[:,0])) < small)[0] # Boundary 1 (RIGHT) - MASTER
    tid2=np.where(abs(XYZ[:,0]-np.amin(XYZ[:,0]))< small)[0] # Boundary 2 (LEFT) - SLAVE
    tid3=np.where(abs(XYZ[:,1]-np.amax(XYZ[:,1]))< small)[0] # Boundary 3 (TOP) - MASTER
    tid4=np.where(abs(XYZ[:,1]-np.amin(XYZ[:,1]))< small)[0] # Boundary 4 (BOTTOM) - SLAVE  

    # Exclude corners from slave node boundaries
    tids2=tid2[1:len(tid2)-1]
    tids4=tid4[1:len(tid4)-1]

    # Get number of slave nodes
    ns2nodes = len(tids2)
    ns4nodes = len(tids4)
    nsnodes = ns2nodes + ns4nodes

    # Create G matrix
    G=np.zeros((2*nsnodes,2*nnodes),dtype=int)          #create an array [DOFs no of slave nodes x DOFs no of all nodes in RVE]

    # loop over slave nodes
    for i in range(nsnodes):

        # First do left boundary
        if i < ns2nodes:
            
            # slave coefficients
            # YOUR CODE HERE - Populate G matrix components with slave node coefficients
            slavenode = ...
    
            # master coefficients
            # YOUR CODE HERE - Populate G matrix components with master node coefficients
            masternode = ...

        # Then top boundary
        else:
            
            # slave coefficients
            # YOUR CODE HERE - Populate G matrix components with slave node coefficients
            slavenode = ...
    
            # master coefficients
            # YOUR CODE HERE - Populate G matrix components with master node coefficients
            masternode = ...

    # Construct slave and master DOFs
    # Slaves
    slavedofs=np.zeros(2*nsnodes,dtype=int);        #array [n x 1] containing total DOF number of slave nodes
    b = np.r_[2*tids4,2*tids2]
    slavedofs[0:2*nsnodes-1:2] = b
    slavedofs[1:2*nsnodes:2] = b + 1

    # Masters, remove slaves from bound corner DOFs
    masterdofs = np.setdiff1d(BCcorner,slavedofs)
    nmnodes = len(masterdofs) // 2

    # Master and slave G matrices
    Gs=G[:,slavedofs]
    Gm=G[:,masterdofs]

    # Transformation matrix
    T=np.zeros((2*nnodes,2*nmnodes),dtype=int)

    # YOUR CODE HERE - Implement transformation matrix
    raise NotImplementedError

    # Remove fixed corners
    T=T[BCcorner,:]
    
    return T

# Compute Solution with PBCs

In [ ]:
# Elasticity matrix
D = plane_strain(E,nu)

# Get boundary conditions (fixed corners and force vector)
BCcorner,f = BC_fun(nxnodes,nynodes,ndofs)

# Periodic boundary transform matrix
T = T_matrix(XYZ,BCcorner)

# Stiffness matrix
K = K_matrix(XYZ,CON,DOF,D,th,BCcorner)

# Modified stiffness matrix with PBCs
# YOUR CODE HERE - Modify stiffness matrix to account for periodic boundary conditions
Km = ...

# YOUR CODE HERE - Modify force matrix to account for periodic boundary conditions
fm = ...

# YOUR CODE HERE - solve for displacement matrix of unconstrained nodes
dm = ...

# YOUR CODE HERE - final modification of displacement vector to account for periodic boundary condition transform
dms = ...

# Global displacement vector
d = np.zeros((ndofs,1))
d[BCcorner] = dms

# Plotting
deformation_plot(XYZ,CON,d)